In [1]:
%pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DataCollatorForTokenClassification

In [3]:
dataset = load_dataset("conll2003")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [4]:
# Define label
label_list = dataset["train"].features["ner_tags"].feature.names

In [5]:
# Tokenizer setup
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Define a function for preprocessing and tokenizing
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True, max_length=128
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        previous_word_id = None
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)
            previous_word_id = word_id
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
# apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

# Create DataLoaders

In [8]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
# Convert the tokenized dataset into Dataloader objects
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
val_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=16, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)


In [10]:
# Check data shape
for batch in train_dataloader:
    print({k: v.shape for k, v in batch.items()})
    break

{'input_ids': torch.Size([16, 128]), 'token_type_ids': torch.Size([16, 128]), 'attention_mask': torch.Size([16, 128]), 'labels': torch.Size([16, 128])}


In [11]:
import torch
import torch.nn as nn
from transformers import AutoModelForTokenClassification, AutoConfig

In [12]:
# Define the NERModel class
class NERModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super(NERModel, self).__init__()

        self.model = AutoModelForTokenClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

        self.loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        if labels is not None:
            loss = self.loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
            return loss, logits
        else:
            return logits

In [13]:
# Set parameters
model_name = "bert-base-cased"
num_labels = len(label_list)

In [14]:
# Instantiate the model
ner_model = NERModel(model_name=model_name, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
ner_model

NERModel(
  (model): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias

In [16]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, accuracy_score

In [17]:
# Parameters for training
num_epochs = 3
batch_size = 16
learning_rate = 5e-5

In [18]:
# DataLoader and model already created from previous steps
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size, collate_fn=data_collator)

In [19]:
# Set up the model, optimizer, and learning rate scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ner_model.to(device)
optimizer = AdamW(ner_model.parameters(), lr=learning_rate)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [20]:
# Training loop
for epoch in range(num_epochs):
    ner_model.train()
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)

    total_loss = 0
    all_preds, all_labels = [], []

    for batch in progress_bar:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = ner_model(input_ids, attention_mask, labels=labels)
        loss, logits = outputs

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

        predictions = torch.argmax(logits, dim=-1)
        all_preds.extend(predictions.view(-1).cpu().numpy())
        all_labels.extend(labels.view(-1).cpu().numpy())

        progress_bar.set_postfix({"loss": loss.item()})

Epoch 1/3:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 2/3:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 3/3:   0%|          | 0/878 [00:00<?, ?it/s]

In [21]:
# Calculate average loss
avg_loss = total_loss / len(train_dataloader)

In [22]:
print(f"Average Training Loss: {avg_loss}")

Average Training Loss: 0.00985917899211877


In [23]:
# Filter out padding tokens for accuracy and f1 calculation
valid_preds = [p for p, l in zip(all_preds, all_labels) if l != -100]
valid_labels = [l for l in all_labels if l != -100]

In [24]:
# Calculate F1 score and accuracy
epoch_f1 = f1_score(valid_labels, valid_preds, average="weighted")
epoch_accuracy = accuracy_score(valid_labels, valid_preds)

In [25]:
    print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {avg_loss:.4f} | F1 Score: {epoch_f1:.4f} | Accuracy: {epoch_accuracy:.4f}")


Epoch 3/3 | Loss: 0.0099 | F1 Score: 0.9975 | Accuracy: 0.9975


# Evaluation

In [26]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

In [30]:
# Define a function for evaluation
def evaluate_model(model, dataloader, label_list):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask)
            logits = outputs

            # Get predictions
            predictions = torch.argmax(logits, dim=-1)

            all_preds.extend(predictions.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    valid_preds = [p for p, l in zip(all_preds, all_labels) if l != -100]
    valid_labels = [l for l in all_labels if l != -100]

    # Calculate precision, recall, and F1-score for each class
    precision = precision_score(valid_labels, valid_preds, average=None, labels=range(len(label_list)))
    recall = recall_score(valid_labels, valid_preds, average=None, labels=range(len(label_list)))
    f1 = f1_score(valid_labels, valid_preds, average=None, labels=range(len(label_list)))

    # Print classification report
    print(classification_report(valid_labels, valid_preds, target_names=label_list))

    return precision, recall, f1

In [31]:
# Run evaluation on validation set
precision, recall, f1 = evaluate_model(ner_model, val_dataloader, label_list)

              precision    recall  f1-score   support

           O       1.00      1.00      1.00     42733
       B-PER       0.97      0.98      0.98      1836
       I-PER       0.99      0.98      0.99      1300
       B-ORG       0.94      0.94      0.94      1341
       I-ORG       0.95      0.92      0.94       751
       B-LOC       0.98      0.97      0.97      1837
       I-LOC       0.94      0.96      0.95       257
      B-MISC       0.92      0.93      0.93       922
      I-MISC       0.87      0.88      0.88       346

    accuracy                           0.99     51323
   macro avg       0.95      0.95      0.95     51323
weighted avg       0.99      0.99      0.99     51323



In [32]:
# Print precision, recall, and F1-score for each label
for i, label in enumerate(label_list):
    print(f"Label: {label}")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Recall: {recall[i]:.4f}")
    print(f"  F1 Score: {f1[i]:.4f}")
    print()

Label: O
  Precision: 0.9976
  Recall: 0.9978
  F1 Score: 0.9977

Label: B-PER
  Precision: 0.9699
  Recall: 0.9837
  F1 Score: 0.9767

Label: I-PER
  Precision: 0.9892
  Recall: 0.9831
  F1 Score: 0.9861

Label: B-ORG
  Precision: 0.9432
  Recall: 0.9403
  F1 Score: 0.9417

Label: I-ORG
  Precision: 0.9480
  Recall: 0.9228
  F1 Score: 0.9352

Label: B-LOC
  Precision: 0.9763
  Recall: 0.9652
  F1 Score: 0.9707

Label: I-LOC
  Precision: 0.9358
  Recall: 0.9650
  F1 Score: 0.9502

Label: B-MISC
  Precision: 0.9237
  Recall: 0.9328
  F1 Score: 0.9282

Label: I-MISC
  Precision: 0.8739
  Recall: 0.8815
  F1 Score: 0.8777



# Early Stopping and Learning Rate Scheduler Setup


In [33]:
from transformers import get_linear_schedule_with_warmup
import numpy as np

In [37]:
# Parameters
num_epochs = 4
patience = 3
best_val_loss = np.inf
no_improve_epochs = 0

In [38]:
# Initialize optimizer and scheduler
optimizer = AdamW(ner_model.parameters(), lr=learning_rate)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [39]:
# Training loop with early stopping and learning rate scheduling
for epoch in range(num_epochs):
    ner_model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    # Training
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = ner_model(input_ids, attention_mask, labels=labels)
        loss, logits = outputs

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

    # Validation
    val_loss = 0
    ner_model.eval()
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = ner_model(input_ids, attention_mask, labels=labels)
            val_loss += outputs[0].item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch + 1} | Training Loss: {total_loss / len(train_dataloader):.4f} | Validation Loss: {avg_val_loss:.4f}")

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1

    if no_improve_epochs >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break

Epoch 1/4:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 1 | Training Loss: 0.0174 | Validation Loss: 0.0465


Epoch 2/4:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 2 | Training Loss: 0.0095 | Validation Loss: 0.0431


Epoch 3/4:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 3 | Training Loss: 0.0032 | Validation Loss: 0.0451


Epoch 4/4:   0%|          | 0/878 [00:00<?, ?it/s]

Epoch 4 | Training Loss: 0.0016 | Validation Loss: 0.0471


# Test

In [40]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [41]:
text = """The Ministry of Education announced at 1 PM on 12th of October 2024 that Perera scored the highest in the Advanced Level examination at Royal College. Captain Cook Restaurant near Lake Gardens is owned by Mr. Silva, who used to teach at Trinity. While Elephant House ice cream remains popular, new brands like Happy Cow from Green Farms Ltd. are gaining market share. The Morning News reported that Dialog's CEO will speak at SLIIT next Friday about Smart Living solutions. The company's annual meeting is scheduled for 1300 10/12/2024 at Unity Plaza. Meanwhile, Unity Plaza shops are preparing for the April Season sales, which coincide with both New Year celebrations. Eagle Insurance (now merged with Union Bank) opened their branch opposite Victoria Park, where Peace Pagoda is clearly visible. Speaking of peace, Peace Cola is launching their new drink called Life, competing with Sprite and Sprint."""


In [46]:
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

In [47]:
# Move the input tensors to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: value.to(device) for key, value in inputs.items()}

In [49]:
# Make predictions
with torch.no_grad():
    ner_model.to(device)
    outputs = ner_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])

    if hasattr(outputs, 'logits'):
        logits = outputs.logits
    else:
        logits = outputs

In [54]:
# Get predictions
predictions = torch.argmax(logits, dim=-1).cpu().numpy().flatten()

In [55]:
# Check unique predicted classes
unique_preds = set(predictions)
print("Unique predicted class indices:", unique_preds)

Unique predicted class indices: {0, 1, 3, 4, 5, 7, 8}


In [56]:
# Update the id2label mapping based on observed indices
id2label = {
    0: "O",
    1: "LOC",
    2: "PER",
    3: "ORG",
    4: "MISC",
    5: "NEW_LABEL_1",
    6: "NEW_LABEL_2",
    7: "NEW_LABEL_3",
    8: "NEW_LABEL_4"
}

In [57]:
entities = []

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].cpu().numpy().flatten())

In [58]:
for token, pred in zip(tokens, predictions):
    if pred in id2label:
        label = id2label[pred]
        if label != "O":  # Only include entities that are recognized
            entities.append((token, label))
    else:
        print(f"Warning: Predicted index {pred} not in id2label mapping.")

In [59]:
# Display identified entities
print("Identified Entities:")
for entity in entities:
    print(entity)

Identified Entities:
('pere', 'NEW_LABEL_1')
('highest', 'ORG')
('in', 'NEW_LABEL_1')
('trinity', 'ORG')
('farms', 'LOC')
('are', 'MISC')
('market', 'NEW_LABEL_1')
('sl', 'NEW_LABEL_3')
('##ii', 'NEW_LABEL_4')
('##t', 'NEW_LABEL_4')
('unity', 'NEW_LABEL_3')
('unity', 'ORG')
('shops', 'ORG')
('are', 'ORG')
('season', 'ORG')
('sales', 'MISC')
('launching', 'LOC')
('called', 'NEW_LABEL_3')
(',', 'NEW_LABEL_3')
('sp', 'NEW_LABEL_4')


In [64]:
# Assuming `predictions` and `tokens` are already defined
for token, pred in zip(tokens, predictions):
    label = id2label.get(pred, "UNKNOWN")  # Use get to avoid KeyError
    print(f"Token: {token}, Predicted Index: {pred}, Label: {label}")

Token: [CLS], Predicted Index: 0, Label: O
Token: the, Predicted Index: 0, Label: O
Token: ministry, Predicted Index: 0, Label: O
Token: of, Predicted Index: 0, Label: O
Token: education, Predicted Index: 0, Label: O
Token: announced, Predicted Index: 0, Label: O
Token: at, Predicted Index: 0, Label: O
Token: 1, Predicted Index: 0, Label: O
Token: pm, Predicted Index: 0, Label: O
Token: on, Predicted Index: 0, Label: O
Token: 12th, Predicted Index: 0, Label: O
Token: of, Predicted Index: 0, Label: O
Token: october, Predicted Index: 0, Label: O
Token: 202, Predicted Index: 0, Label: O
Token: ##4, Predicted Index: 0, Label: O
Token: that, Predicted Index: 0, Label: O
Token: pere, Predicted Index: 5, Label: NEW_LABEL_1
Token: ##ra, Predicted Index: 0, Label: O
Token: scored, Predicted Index: 0, Label: O
Token: the, Predicted Index: 0, Label: O
Token: highest, Predicted Index: 3, Label: ORG
Token: in, Predicted Index: 5, Label: NEW_LABEL_1
Token: the, Predicted Index: 0, Label: O
Token: ad

In [66]:
class NERModel(torch.nn.Module):
    def __init__(self, model_name, num_labels):
        super(NERModel, self).__init__()
        self.bert = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs  # This should include logits


In [67]:
with torch.no_grad():
    ner_model.to(device)
    outputs = ner_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs  # If outputs is a tensor


In [68]:
ner_model = NERModel(model_name="bert-base-cased", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
with torch.no_grad():
    ner_model.to(device)
    outputs = ner_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    # Check the type of outputs
    if isinstance(outputs, torch.Tensor):
        logits = outputs
    else:
        logits = outputs.logits

# Get predictions
predictions = torch.argmax(logits, dim=-1).cpu().numpy().flatten()

# Define your id2label mapping
id2label = {
    0: "O",        # Outside
    1: "LOC",      # Location
    2: "PER",      # Person
    3: "ORG",      # Organization
    4: "MISC",     # Miscellaneous
}

# Extract tokens
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].cpu().numpy().flatten())

for token, pred in zip(tokens, predictions):
    label = id2label.get(pred, "UNKNOWN")
    print(f"Token: {token}, Predicted Index: {pred}, Label: {label}")

person_names = [token for token, pred in zip(tokens, predictions) if id2label.get(pred) == "PER"]

print("Identified Person Names:")
if person_names:
    for name in person_names:
        print(name)
else:
    print("No person names identified.")


Token: [CLS], Predicted Index: 7, Label: UNKNOWN
Token: the, Predicted Index: 7, Label: UNKNOWN
Token: ministry, Predicted Index: 7, Label: UNKNOWN
Token: of, Predicted Index: 2, Label: PER
Token: education, Predicted Index: 7, Label: UNKNOWN
Token: announced, Predicted Index: 7, Label: UNKNOWN
Token: at, Predicted Index: 0, Label: O
Token: 1, Predicted Index: 7, Label: UNKNOWN
Token: pm, Predicted Index: 7, Label: UNKNOWN
Token: on, Predicted Index: 0, Label: O
Token: 12th, Predicted Index: 7, Label: UNKNOWN
Token: of, Predicted Index: 2, Label: PER
Token: october, Predicted Index: 7, Label: UNKNOWN
Token: 202, Predicted Index: 7, Label: UNKNOWN
Token: ##4, Predicted Index: 7, Label: UNKNOWN
Token: that, Predicted Index: 0, Label: O
Token: pere, Predicted Index: 6, Label: UNKNOWN
Token: ##ra, Predicted Index: 7, Label: UNKNOWN
Token: scored, Predicted Index: 2, Label: PER
Token: the, Predicted Index: 4, Label: MISC
Token: highest, Predicted Index: 0, Label: O
Token: in, Predicted Inde